In [1]:
import json
import time

import parsivar

# Phase 2

## Pre Phase

In [2]:
# Loading File in Json format
f = open('IR_data_news_12k.json')
data = json.load(f)

In [3]:
# initializing library classes
normalizer = parsivar.Normalizer()
tokenizer = parsivar.Tokenizer()
stemmer = parsivar.FindStems()

In [4]:
# loading stopwords list from external file
stopWordsList = open('stopWords.txt').read().splitlines()

In [5]:
# used to save docsInfo
processedDocs = {}
allDocsCount = len(data)

In [6]:
for i in range(allDocsCount):
    # normalizing and tokenizing the content of a file
    #[:-15] is to remove "انتهای پیام /"
    tokens = tokenizer.tokenize_words(normalizer.normalize(data[str(i)]['content'][:-15]))

    # used to store valuable tokens
    processedTokens = []
    for token in tokens:
        # checks for stop words
        if token not in stopWordsList:
            # stemming tokens
            stemmed = stemmer.convert_to_stem(token)
            processedTokens.append(stemmed)
    # storing tuple of (title, url, tokens) of a document with key "id"
    processedDocs[i] = (data[str(i)]['title'], data[str(i)]['url'], processedTokens)

## SubSection 1

In [7]:
import structlinks
from structlinks import LinkedList
import math

In [8]:
docsMap = {}
positionalIndex = {}
docSize = {}

In [9]:
for i in processedDocs.keys():
    if i % 1000 == 0:
        print(i)
    title, url, processedTokens = processedDocs[i]

    docsMap[i] = (title, url)
    docSize[i] = len(processedTokens)

    pos = 0
    for term in processedTokens:
        if term in positionalIndex.keys():
            allNum, termDocList = positionalIndex[term]

            # j = 0
            # for doc in termDocList:
            #     docID, docNum, docPosList = doc
            #     if i == docID:
            #
            #         # PosPos = len(docPosList)
            #         # for k in range(len(docPosList)):
            #         #     if docPosList[k] > pos:
            #         #         PosPos = k
            #         # docPosList.insert(PosPos, pos)
            #
            #         docPosList.append(pos)
            #
            #         docNum += 1
            #         termDocList[j] = (docID, docNum, docPosList)
            #
            #         allNum += 1
            #         positionalIndex[term] = (allNum, termDocList)
            #         break
            #     j += 1

            docID, docNum, tf_idf, docPosList = termDocList[-1]
            if docID == i:
                docPosList.append(pos)
                docNum += 1
                termDocList[-1] = (docID, docNum, 0, docPosList)

                allNum += 1
                positionalIndex[term] = (allNum, termDocList)
            else:
                docPosList = LinkedList.LinkedList([pos])

                # docPos = len(termDocList)
                # for k in range(len(termDocList)):
                #     docID, docNum, TempdocPosList = termDocList[k]
                #     if docID > i:
                #         docPos = k
                # termDocList.insert(docPos, (i, 1, docPosList))

                termDocList.append((i, 1, 0, docPosList))

                allNum += 1
                positionalIndex[term] = (allNum, termDocList)

        else:
            docPosList = LinkedList.LinkedList([pos])
            termDocList = LinkedList.LinkedList([(i, 1, 0, docPosList)])
            positionalIndex[term] = (1, termDocList)
        pos += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000


In [10]:
import time

In [11]:
lengths = {}
idfs = {}

In [57]:
count = 0
st = time.time()

for term in positionalIndex.keys():
    count += 1
    if count % 100 == 0:
        print(count)

    allNum, termDocList = positionalIndex[term]
    docFrequency = len(termDocList)
    idf = math.log10(allDocsCount/ docFrequency)
    idfs[term] = idf
    for i in range(docFrequency):
        docID, docNum, tf_idf, docPosList = termDocList[i]

        tf = (1 + math.log10(docNum))

        termDocList[i] = (docID, docNum, tf * idf, docPosList)
        lengths[docID] += (tf * idf)**2
    positionalIndex[term] = (allNum, termDocList)

print(time.time() - st)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [13]:
allNum, termDocList = positionalIndex['گزارش']
print("_______________________________________________________")
print('Term : ', 'گزارش')
print('Frequency in all Docs :', allNum)
print('Docs : ')
for doc in termDocList:
    docID, docNum, tf_idf, docPosList = doc
    print("***********************")
    print('docID : ', docID)
    print('Frequency in this Doc :', docNum)
    print('positions : ', docPosList)
    print('tf_idf', tf_idf)

_______________________________________________________
Term :  گزارش
Frequency in all Docs : 14013
Docs : 
***********************
docID :  0
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  1
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  2
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  3
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  4
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  5
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  6
Frequency in this Doc : 2
positions :  [0 -> 54]
tf_idf 0.0858508910618606
***********************
docID :  7
Frequency in this Doc : 1
positions :  [0]
tf_idf 0.06598686528979415
***********************
docID :  8
Frequency in

## SubSection 2

In [14]:
def printDocs(docList):
    print("###################################################################")
    print("Number of Results : ", len(docList))
    for doc in docList[:5]:
        docID, score, sentences = doc
        title, url = docsMap[docID]
        print("*************************************************************************************")
        print('docID : ', docID)
        print('title : ', title)
        print('url : ', url)
        print('score :', score)

        for term in sentences.keys():
            print(term, " : ")
            for s in sentences[term]:
                print(s)
                print("___________________________________________________")

In [15]:
def retreiveSentences(docID, terms):
    result = {}
    sentences = tokenizer.tokenize_sentences(normalizer.normalize(data[str(docID)]['content'][:-15]))


    for term in terms:
        for sentence in sentences:
            if term in sentence:
                if term in result.keys() :
                    result[term].append(sentence)
                else:
                    result[term] = [sentence]

    return result

In [44]:
def processQueryCosine(termsWeights):
    docs = {}

    for term in termsWeights.keys():
        wt = (1 + math.log10(termsWeights[term]))
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in positionalIndex.keys()):
            termDocsList = positionalIndex[stemmed][1]
            for i in range(len(termDocsList)):
                docID, docNum, tf_idf, docPosList = termDocsList[i]
                if docID in docs:
                    docID, score, sentences = docs[docID]
                    score += tf_idf * wt
                    docs[docID] = (docID, score, sentences)
                else:
                    if (docID == 84):
                    sentences = retreiveSentences(docID, termsWeights.keys())
                    docs[docID] = (docID, tf_idf * wt, sentences)
        else:
            print("Couldn't find", stemmed)

    for docID in docs.keys():
        docID, score, sentences = docs[docID]
        score /= math.sqrt(lengths[docID])
        docs[docID] = (docID, score, sentences)

    rankedDocs = sorted(docs.values(), key= lambda x: -x[1])

    return rankedDocs

In [33]:
def processQueryJacquard(termsWeights):
    docs = {}
    querySize = len(termsWeights.keys())
    for termFreq in termsWeights.values():
        querySize += termFreq

    for term in termsWeights.keys():
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in positionalIndex.keys()):
            termDocsList = positionalIndex[stemmed][1]
            for i in range(len(termDocsList)):
                docID, docNum, tf_idf, docPosList = termDocsList[i]
                docSet = set(processedDocs[docID][2])
                docLen = len(docSet)
                jacquard = (1) / (querySize + docLen)
                if docID in docs:
                    docID, score, sentences = docs[docID]
                    score += jacquard
                    docs[docID] = (docID, score, sentences)
                else:
                    sentences = retreiveSentences(docID, termsWeights.keys())
                    docs[docID] = (docID, jacquard, sentences)
        else:
            print("Couldn't find", stemmed)

    rankedDocs = sorted(docs.values(), key= lambda x: -x[1])

    return rankedDocs

In [18]:
def preProcessQuery (input):
    terms = input.split(" ")
    termWeights = {}

    for term in terms:
        if term not in stopWordsList:
            if term in termWeights.keys():
                termWeights[term] += 1
            else:
                termWeights[term] = 1

    return termWeights

In [58]:
termsWeights = preProcessQuery('فوتسال')
results = processQueryCosine(termsWeights)

if len(results) == 0:
    print('none')
else:
    printDocs(results)

score 84 : 3.310428682395911
tf_idf 84 : 3.310428682395911
wt : فوتسال : 1.0
0.26964504327163513
###################################################################
Number of Results :  196
*************************************************************************************
docID :  5073
title :  تیم ملی و ناظم الشریعه نامزد برترین های فوتسال جهان
url :  https://www.farsnews.ir/news/14001019000727/تیم-ملی-و-ناظم-الشریعه-نامزد-برترین-های-فوتسال-جهان
score : 0.3383029076235488
فوتسال  : 
به ‌گزارش خبرنگار ورزش فارس ، سایت رسمی فوتسال پلنت که هر ساله برترین‌های فوتسال جهان را معرفی می‌کند در سال 2021 هم برترین‌های فوتسال جهان را معرفی کرد  .
___________________________________________________
 نام سید محمد ناظم الشریعه سرمربی تیم‌ملی فوتسال کشورمان و همچنین تیم ملی فوتسال ایران جزو 10 گزینه نهایی انتخاب برترینهای فوتسال جهان قرار گرفته است  .
___________________________________________________
*************************************************************************************
docID :  

In [35]:
termsWeights = preProcessQuery('باشگاه والیبال')
results = processQueryJacquard(termsWeights)

if len(results) == 0:
    print('none')
else:
    printDocs(results)

###################################################################
Number of Results :  2513
*************************************************************************************
docID :  833
title :  ستاره بلژیکی بهترین بازیکن ماه رئال مادرید
url :  https://www.farsnews.ir/news/14001213000563/ستاره-بلژیکی-بهترین-بازیکن-ماه-رئال-مادرید
score : 0.045454545454545456
باشگاه  : 
به گزارش خبرگزاری فارس ، تیبو کورتوا سنگربان بلژیکی باشگاه رئال مادرید به عنوان بهترین ماه فوریه باشگاه رئال مادرید انتخاب شد  .
___________________________________________________
*************************************************************************************
docID :  1116
title :  میزبان مسابقات والیبال جام باشگاه‌های آسیا مشخص شد
url :  https://www.farsnews.ir/news/14001209000462/میزبان-مسابقات-والیبال-جام-باشگاه‌های-آسیا-مشخص-شد
score : 0.044444444444444446
باشگاه  : 
به گزارش خبرنگار ورزشی خبرگزاری فارس ، پس از اعلام انصراف ارومیه از میزبانی مسابقات والیبال جام باشگاه‌های آسیا ، شهرهای تهران و اصفهان به

## SubSection 3

In [21]:
def createChampionList (r) :
    enahncedPositionalIndex = {}
    count = 0
    st = time.time()

    for term in positionalIndex.keys():
        count += 1
        if count % 100 == 0:
            print(count)

        allNum, termDocList = positionalIndex[term]
        championList = LinkedList.LinkedList()

        for i in range(len(termDocList)):
            docID, docNum, tf_idf, docPosList = termDocList[i]

            inserted = False
            for i, (TdocID, TdocNum, Ttf_idf, TdocPosList) in enumerate(championList):
                if tf_idf > Ttf_idf:
                    championList.insert(i, (docID, docNum, tf_idf, docPosList))
                    inserted = True
                    break

            if not inserted and len(championList) < r:
                championList.append((docID, docNum, tf_idf, docPosList))
                inserted = True

            if inserted and len(championList) >= r:
                championList.pop(-1)

        enahncedPositionalIndex[term] = (allNum, championList)

    print(time.time() - st)
    return enahncedPositionalIndex

In [22]:
enhancedPositionalIndex = createChampionList(50)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [23]:
def preProcessQueryEnhanced(input, idfT):
    terms = input.split(" ")
    termWeights = {}

    for term in terms:
        if term not in stopWordsList:
            if idfs[term] > idfT:
                if term in termWeights.keys():
                    termWeights[term] += 1
                else:
                    termWeights[term] = 1

    return termWeights

In [24]:
def processQueryCosineEnhanced(termsWeights, n):
    preDocs = {}

    for term in termsWeights.keys():
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in enhancedPositionalIndex.keys()):
            championList = enhancedPositionalIndex[stemmed][1]
            for i in range(len(championList)):
                docID, docNum, tf_idf, docPosList = championList[i]
                if docID in preDocs:
                    docID, freq = preDocs[docID]
                    freq += 1
                    preDocs[docID] = (docID, freq)
                else:
                    preDocs[docID] = (docID, 1)
        else:
            print("Couldn't find", stemmed)

    filteredDocs = set()
    for docID, freq in preDocs.values():
        if freq > n:
            filteredDocs.add(docID)

    docs = {}

    for term in termsWeights.keys():
        wt = (1 + math.log10(termsWeights[term]))
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in enhancedPositionalIndex.keys()):
            championList = enhancedPositionalIndex[stemmed][1]
            for i in range(len(championList)):
                docID, docNum, tf_idf, docPosList = championList[i]
                if docID in filteredDocs:
                    if docID in docs:
                        docID, score, sentences = docs[docID]
                        score += tf_idf * wt
                        docs[docID] = (docID, score, sentences)
                    else:
                        sentences = retreiveSentences(docID, termsWeights.keys())
                        docs[docID] = (docID, tf_idf * wt, sentences)

    for docID in docs.keys():
        docID, score, sentences = docs[docID]
        score /= math.sqrt(lengths[docID])
        docs[docID] = (docID, score, sentences)

    rankedDocs = sorted(docs.values(), key= lambda x: -x[1])

    return rankedDocs

In [59]:
def processQueryJacquardEnhanced(termsWeights, n):
    preDocs = {}

    for term in termsWeights.keys():
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in enhancedPositionalIndex.keys()):
            championList = enhancedPositionalIndex[stemmed][1]
            for i in range(len(championList)):
                docID, docNum, tf_idf, docPosList = championList[i]
                if docID in preDocs:
                    docID, freq = preDocs[docID]
                    freq += 1
                    preDocs[docID] = (docID, freq)
                else:
                    preDocs[docID] = (docID, 1)
        else:
            print("Couldn't find", stemmed)

    filteredDocs = set()
    for docID, freq in preDocs.values():
        if freq > n:
            filteredDocs.add(docID)

    docs = {}
    querySize = len(termsWeights.keys())

    for term in termsWeights.keys():
        stemmed = stemmer.convert_to_stem(term)
        if (stemmed in enhancedPositionalIndex.keys()):
            championList = enhancedPositionalIndex[stemmed][1]
            for i in range(len(championList)):
                docID, docNum, tf_idf, docPosList = championList[i]
                if docID in filteredDocs:
                    docSet = set(processedDocs[docID][2])
                    docLen = len(docSet)
                    jacquard = (1) / (querySize + docLen)
                    if docID in docs:
                        docID, score, sentences = docs[docID]
                        score += jacquard
                        docs[docID] = (docID, score, sentences)
                    else:
                        sentences = retreiveSentences(docID, termsWeights.keys())
                        docs[docID] = (docID, jacquard, sentences)

    rankedDocs = sorted(docs.values(), key= lambda x: -x[1])

    return rankedDocs

In [26]:
termsWeights = preProcessQueryEnhanced('مایکل جردن', 0.5)
results = processQueryCosineEnhanced(termsWeights, 1)

if len(results) == 0:
    print('none')
else:
    printDocs(results)

###################################################################
Number of Results :  1
*************************************************************************************
docID :  85
title :  با پردرآمدترین ورزشکاران جهان آشنا شوید؛ از MMA تا فوتبال و بسکتبال
url :  https://www.farsnews.ir/news/14001223000292/با-پردرآمدترین-ورزشکاران-جهان-آشنا-شوید-از-MMA-تا-فوتبال-و-بسکتبال
score : 1.828676476282413
مایکل  : 
 او بارها به عنوان بهترین بازیکن تاریخ بسکتبال شناخته‌شده‌و اغلب با مایکل جردن مقایسه می‌شود  .
___________________________________________________
جردن  : 
 او بارها به عنوان بهترین بازیکن تاریخ بسکتبال شناخته‌شده‌و اغلب با مایکل جردن مقایسه می‌شود  .
___________________________________________________


In [60]:
termsWeights = preProcessQueryEnhanced('مایکل جردن', 0.5)
results = processQueryJacquardEnhanced(termsWeights, 1)

if len(results) == 0:
    print('none')
else:
    printDocs(results)

###################################################################
Number of Results :  1
*************************************************************************************
docID :  85
title :  با پردرآمدترین ورزشکاران جهان آشنا شوید؛ از MMA تا فوتبال و بسکتبال
url :  https://www.farsnews.ir/news/14001223000292/با-پردرآمدترین-ورزشکاران-جهان-آشنا-شوید-از-MMA-تا-فوتبال-و-بسکتبال
score : 0.010362694300518135
مایکل  : 
 او بارها به عنوان بهترین بازیکن تاریخ بسکتبال شناخته‌شده‌و اغلب با مایکل جردن مقایسه می‌شود  .
___________________________________________________
جردن  : 
 او بارها به عنوان بهترین بازیکن تاریخ بسکتبال شناخته‌شده‌و اغلب با مایکل جردن مقایسه می‌شود  .
___________________________________________________


In [37]:
print(idfs['فوتسال'])

1.7941749492998926


In [56]:
allNum, termDocList = positionalIndex['فوتسال']
docFrequency = len(termDocList)
print(docFrequency)
idf = math.log10(allDocsCount/ docFrequency)
print(idf)
for i in range(len(termDocList)):
    docID, docNum, tf_idf, docPosList = termDocList[i]
    if docID == 84:
        tf = (1 + math.log10(docNum))
        print(tf)

196
1.7941749492998926
1.845098040014257
